In [1]:
import pandas as pd
from src.train import train_loop, test_loop
from src.model import CNNLSTMModel
from src.dataset import InverterTimeSeriesDataset
from src.preprocess import *
import torch

inverter_data = load_parquet_data('data/inverter_data')
failure_sessions = load_failure_sessions('data/failure_sessions.csv', min_days=3)

Loaded 15 parquet files → 6126272 rows
Kept 61 sessions longer than 3 days


In [2]:
labeled_df = prepare_dataset(inverter_data, failure_sessions)
labeled_df.fillna(0, inplace=True)

Total pre-failure rows: 82486
Total rows: 5905370


In [3]:
train_df, test_df = train_test_split_on_time(labeled_df, 0.2)
val_df, test_df = train_test_split_on_time(test_df, 0.5)

Train set size: 4724296 Train set time range: 2021-12-02 00:00:00 to 2024-10-27 01:00:00
Test set size: 1181074 Test set time range: 2024-10-27 01:00:00 to 2025-07-23 23:35:00
Train set size: 590537 Train set time range: 2024-10-27 01:00:00 to 2025-03-04 04:40:00
Test set size: 590537 Test set time range: 2025-03-04 04:40:00 to 2025-07-23 23:35:00


In [4]:
feature_cols = [
    "metric.AC_POWER.MEASURED",
    "metric.DC_POWER.MEASURED",
    "metric.AC_CURRENT_A.MEASURED",
    "metric.AC_CURRENT_B.MEASURED",
    "metric.AC_CURRENT_C.MEASURED",
    "metric.DC_CURRENT.MEASURED",
    "metric.DC_CURRENT_AVG.MEASURED",
    "metric.DC_CURRENT_MAX.MEASURED",
    "metric.FREQUENCY.MEASURED",
    "metric.POWER_FACTOR.MEASURED",
    "metric.HEARTBEAT.MEASURED",
    "metric.COMM_LINK.MEASURED",
    "metric.STATUS_WARNING_WORD.MEASURED",
    "metric.STATUS_FAULT_WORD.MEASURED",
    "metric.STATUS_IGBT_MAX_TEMP.MEASURED",
    "metric.STATUS_INTERNAL_TEMP.MEASURED",
    "metric.STATUS_INTERNAL_HUMIDITY.MEASURED"
]


from torch.utils.data import DataLoader

train_ds = InverterTimeSeriesDataset(train_df, feature_cols, under_sample=True)
val_ds   = InverterTimeSeriesDataset(val_df,   feature_cols)
test_ds  = InverterTimeSeriesDataset(test_df,  feature_cols)

Processing devices: 100%|██████████| 16/16 [00:01<00:00,  8.40it/s]


In [5]:
batch_size = 2**13
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=6, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=6, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, num_workers=6, pin_memory=True)

In [17]:
model = CNNLSTMModel(num_features=len(feature_cols))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.MSELoss()

In [18]:
train_loop(model, train_loader, val_loader, log_interval=10, num_epochs=30, optimizer=optimizer, criterion=criterion)

Model moved to cuda
[Epoch 1/30] Step 0/16 - Loss: 0.2503
[Epoch 1/30] Step 10/16 - Loss: 0.2490
🔁 Epoch 1 finished. Avg Train Loss: 0.2492
✅ Validation Loss: 0.2290 | Accuracy: 76.87%
[Epoch 2/30] Step 0/16 - Loss: 0.2468
[Epoch 2/30] Step 10/16 - Loss: 0.2455
🔁 Epoch 2 finished. Avg Train Loss: 0.2461
✅ Validation Loss: 0.2583 | Accuracy: 60.30%
[Epoch 3/30] Step 0/16 - Loss: 0.2417
[Epoch 3/30] Step 10/16 - Loss: 0.2397
🔁 Epoch 3 finished. Avg Train Loss: 0.2408
✅ Validation Loss: 0.2361 | Accuracy: 70.33%
[Epoch 4/30] Step 0/16 - Loss: 0.2376
[Epoch 4/30] Step 10/16 - Loss: 0.2359
🔁 Epoch 4 finished. Avg Train Loss: 0.2367
✅ Validation Loss: 0.2080 | Accuracy: 77.57%
[Epoch 5/30] Step 0/16 - Loss: 0.2341
[Epoch 5/30] Step 10/16 - Loss: 0.2346
🔁 Epoch 5 finished. Avg Train Loss: 0.2350
✅ Validation Loss: 0.1972 | Accuracy: 79.28%
[Epoch 6/30] Step 0/16 - Loss: 0.2327
[Epoch 6/30] Step 10/16 - Loss: 0.2348
🔁 Epoch 6 finished. Avg Train Loss: 0.2341
✅ Validation Loss: 0.2200 | Accurac

In [23]:
train_loop(model, train_loader, val_loader, log_interval=10, num_epochs=30, optimizer=optimizer, criterion=criterion)

Model moved to cuda
[Epoch 1/30] Step 0/16 - Loss: 0.2068
[Epoch 1/30] Step 10/16 - Loss: 0.2066
🔁 Epoch 1 finished. Avg Train Loss: 0.2076
✅ Validation Loss: 0.1820 | Accuracy: 73.55%
[Epoch 2/30] Step 0/16 - Loss: 0.2145
[Epoch 2/30] Step 10/16 - Loss: 0.2057
🔁 Epoch 2 finished. Avg Train Loss: 0.2079
✅ Validation Loss: 0.1995 | Accuracy: 65.93%
[Epoch 3/30] Step 0/16 - Loss: 0.2070
[Epoch 3/30] Step 10/16 - Loss: 0.2046
🔁 Epoch 3 finished. Avg Train Loss: 0.2057
✅ Validation Loss: 0.1771 | Accuracy: 68.40%
[Epoch 4/30] Step 0/16 - Loss: 0.2021
[Epoch 4/30] Step 10/16 - Loss: 0.2012
🔁 Epoch 4 finished. Avg Train Loss: 0.2028
✅ Validation Loss: 0.2028 | Accuracy: 64.39%
[Epoch 5/30] Step 0/16 - Loss: 0.2036
[Epoch 5/30] Step 10/16 - Loss: 0.2058
🔁 Epoch 5 finished. Avg Train Loss: 0.2045
✅ Validation Loss: 0.1859 | Accuracy: 71.78%
[Epoch 6/30] Step 0/16 - Loss: 0.2039
[Epoch 6/30] Step 10/16 - Loss: 0.2022
🔁 Epoch 6 finished. Avg Train Loss: 0.2027
✅ Validation Loss: 0.1806 | Accurac

In [28]:
train_loop(model, train_loader, val_loader, log_interval=10, num_epochs=30, optimizer=optimizer, criterion=criterion)

Model moved to cuda
[Epoch 1/30] Step 0/16 - Loss: 0.1830
[Epoch 1/30] Step 10/16 - Loss: 0.1862
🔁 Epoch 1 finished. Avg Train Loss: 0.1853
✅ Validation Loss: 0.1446 | Accuracy: 72.35%
[Epoch 2/30] Step 0/16 - Loss: 0.1861
[Epoch 2/30] Step 10/16 - Loss: 0.1855
🔁 Epoch 2 finished. Avg Train Loss: 0.1862
✅ Validation Loss: 0.1338 | Accuracy: 77.18%
[Epoch 3/30] Step 0/16 - Loss: 0.1782
[Epoch 3/30] Step 10/16 - Loss: 0.1847
🔁 Epoch 3 finished. Avg Train Loss: 0.1833
✅ Validation Loss: 0.1448 | Accuracy: 73.71%
[Epoch 4/30] Step 0/16 - Loss: 0.1844
[Epoch 4/30] Step 10/16 - Loss: 0.1827
🔁 Epoch 4 finished. Avg Train Loss: 0.1832
✅ Validation Loss: 0.1441 | Accuracy: 73.55%
[Epoch 5/30] Step 0/16 - Loss: 0.1846
[Epoch 5/30] Step 10/16 - Loss: 0.1809
🔁 Epoch 5 finished. Avg Train Loss: 0.1815
✅ Validation Loss: 0.1525 | Accuracy: 74.39%
[Epoch 6/30] Step 0/16 - Loss: 0.1783
[Epoch 6/30] Step 10/16 - Loss: 0.1856
🔁 Epoch 6 finished. Avg Train Loss: 0.1834
✅ Validation Loss: 0.1538 | Accurac

In [ ]:
trues, predictions = test_loop(model, test_loader, device='cuda', criterion=criterion)

🔍 Test Loss: 0.1549 | Accuracy: 75.69%


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(trues, predictions , target_names=['Normal', 'Failure']))
print(confusion_matrix(trues, predictions ))

              precision    recall  f1-score   support

      Normal       0.97      0.77      0.86    571737
     Failure       0.03      0.21      0.05     17495

    accuracy                           0.76    589232
   macro avg       0.50      0.49      0.45    589232
weighted avg       0.94      0.76      0.84    589232

[[442299 129438]
 [ 13828   3667]]
